Goal: find the best model to detect and classify seizures and other types of harmful brain activity. The model was trained on electroencephalography (EEG) signals recorded from critically ill hospital patients.
Dataset: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/data
Steps:
* 1. Spectrum data normalization and transformation.
* 2. Train and test the model (split the dataset into 80/20 for training and testing and further split the 80 into 80/20 for training and validation)
* 2.1 EfficientNet
* 2.2 ResNet
* 2.3 DenseNet
* 3. Ensemble the three models to predict.
* 4. Result: 0.53


In [5]:
print('Initialize ENV')
RUN_TRAINED_MODEL = False
RUN_ON_KAGGLE = False
LOAD_KAGGLE_SPECTROGRAMS = False
FOR_SUBMISSION = False
MIX = True

Initialize ENV


In [6]:
print('import libs')
import warnings
warnings.filterwarnings("ignore")
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import time
import math
import random
import json
import tensorflow as tf
import pandas as pd, numpy as np

import libs


In [7]:
print('Config GPU')
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)<=1: 
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f'Using {len(gpus)} GPU')
else: 
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} GPUs')


Config GPU
Using 0 GPU


In [8]:
print('Mix Precision')
if MIX:
    tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
    print('Mixed precision enabled')
else:
    print('Using full precision')

Mix Precision
Mixed precision enabled


In [9]:
print('set up file path')
if RUN_ON_KAGGLE:
    print('run on kaggle!!!')
    FILE_BASE_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'
    MY_FILE_BASE_PATH = '/kaggle/input/hms-egg-spectrograms/'
    #MY_FILE_BASE_PATH = '/kaggle/input/hms-eeg-sptrograms-fd/'
else:
    FILE_BASE_PATH = 'C:\\Work\\projects\\ML\\jupyter_book\\HMS - Harmful Brain Activity Classification\\'
    MY_FILE_BASE_PATH = 'C:\\Work\\projects\\ML\\jupyter_book\\HMS - Harmful Brain Activity Classification\\'

set up file path
run on local!!!


In [13]:
print('load train.csv')
df = pd.read_csv(FILE_BASE_PATH + "train.csv")
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

load train.csv


,isic_id,target
0,ISIC_0015670,0
1,ISIC_0015845,0
2,ISIC_0015864,0
3,ISIC_0015902,0
4,ISIC_0024200,0


In [7]:
print('Select Trained Data and Labels')
if not RUN_TRAINED_MODEL:
    #Only use train dat with total vote count > 10
    df1 = df[df['seizure_vote'] + df['lpd_vote'] + df['gpd_vote'] + 
             df['lrda_vote'] + df['grda_vote'] + df['other_vote'] > 10]

    train = df1.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
        {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
    train.columns = ['spec_id','min']

    tmp = df1.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
        {'spectrogram_label_offset_seconds':'max'})
    train['max'] = tmp

    tmp = df1.groupby('eeg_id')[TARGETS].agg('sum')
    for t in TARGETS:
        train[t] = tmp[t].values

    y_data = train[TARGETS].values
    y_data = y_data / y_data.sum(axis=1,keepdims=True)
    train[TARGETS] = y_data

    tmp = df1.groupby('eeg_id')[['expert_consensus']].agg('first')
    train['target'] = tmp

    train = train.reset_index()
    print('Train non-overlapp eeg_id shape:', train.shape )

    train.head()

Select Trained Data and Labels
Train non-overlapp eeg_id shape: (5841, 11)


In [9]:
%%time
print('Load Train Spectrograms')
if not RUN_TRAINED_MODEL:
    if LOAD_KAGGLE_SPECTROGRAMS:
        if RUN_ON_KAGGLE:
            TRAIN_SPEC_PATH = FILE_BASE_PATH + "train_spectrograms/"
        else: 
            TRAIN_SPEC_PATH = FILE_BASE_PATH + "train_spectrograms\\"

        files = os.listdir(TRAIN_SPEC_PATH)
        print(f'There are {len(files)} spectrogram parquets')

        spectrograms_ids = np.empty((0), dtype = np.int64)
        spectrograms_images = np.empty((0, 256, 400))
        spectrograms_labels = np.empty((0,6))
        spectrograms_experts = np.empty((0), dtype = np.string_)
        for i,f in enumerate(files):
            if i%100==0:print(i,', ',end='')
            name = int(f.split('.')[0])
            tmp2 = train[train["spec_id"]==name]
            if tmp2.shape[0] > 0:
                tmp = pd.read_parquet(f'{TRAIN_SPEC_PATH}{f}')
                spectrograms_labels = np.insert(spectrograms_labels, spectrograms_labels.shape[0], 
                                                    tmp2[0:1][TARGETS], axis=0)
                spectrograms_images = np.insert(spectrograms_images, spectrograms_images.shape[0], 
                                                    tmp.iloc[0:256,1:].values, axis=0)
                spectrograms_ids = np.insert(spectrograms_ids, spectrograms_ids.shape[0], [name], axis=0)
                spectrograms_experts = np.insert(spectrograms_experts, spectrograms_experts.shape[0], 
                                                    tmp2[0:1]["target"], axis=0)
        if not FOR_SUBMISSION:
            with open('spectrograms_labels.npy', 'wb') as fspectrograms_labels:
                np.save(fspectrograms_labels, spectrograms_labels)

            with open('spectrograms_images.npy', 'wb') as fspectrograms_images:
                np.save(fspectrograms_images, spectrograms_images)

            with open('spectrograms_ids.npy', 'wb') as fspectrograms_ids:
                np.save(fspectrograms_ids, spectrograms_ids)

            with open('spectrograms_experts.npy', 'wb') as fspectrograms_experts:
                np.save(fspectrograms_experts, spectrograms_experts)
            
    else:
        with open(MY_FILE_BASE_PATH + 'spectrograms_labels.npy', 'rb') as fspectrograms_labels:
            spectrograms_labels = np.load(fspectrograms_labels)
    
        with open(MY_FILE_BASE_PATH + 'spectrograms_images.npy', 'rb') as fspectrograms_images:
            spectrograms_images = np.load(fspectrograms_images)
    
        with open(MY_FILE_BASE_PATH + 'spectrograms_ids.npy', 'rb') as fspectrograms_ids:
            spectrograms_ids = np.load(fspectrograms_ids)
    
        with open(MY_FILE_BASE_PATH + 'spectrograms_experts.npy', 'rb') as fspectrograms_experts:
            spectrograms_experts = np.load(fspectrograms_experts)

Load Train Spectrograms
CPU times: total: 3.36 s
Wall time: 12 s


In [10]:
print('re-size spectrograms from 256x400x1 to 256x100x4')
def egg_spec_img_resize(spec_imags):
    #spec_images_c3 = np.zeros((spec_imags.shape[0], 64, 200, 3), dtype='float32')
    
    #spec_images_c3[:,:, 0:200, 0] = spec_imags[:,96:160, 0:200]
    #spec_images_c3[:,:, 0:200, 1] = spec_imags[:,96:160, 200:400]
    #spec_images_c3[:,:, 0:100, 2] = spec_imags[:,96:160, 100:200]
    #spec_images_c3[:,:, 100:200, 2] = spec_imags[:,96:160, 300:400]
    
    spec_images_c3 = np.zeros((spec_imags.shape[0], 128, 200, 3), dtype='float32')
    
    spec_images_c3[:,:, 0:200, 0] = spec_imags[:,64:192, 0:200]
    spec_images_c3[:,:, 0:200, 1] = spec_imags[:,64:192, 200:400]
    spec_images_c3[:,:, 0:100, 2] = spec_imags[:,64:192, 100:200]
    spec_images_c3[:,:, 100:200, 2] = spec_imags[:,64:192, 300:400]
    
    return spec_images_c3

if not RUN_TRAINED_MODEL:
    spectrograms_images_c3 = egg_spec_img_resize(spectrograms_images)
    del spectrograms_images
    gc.collect()

re-size spectrograms from 256x400x1 to 256x100x4


In [11]:
print('Normaliztion spectrogram values to 0 ... 255')    
def egg_spec_img_normalization(spec_images):
    # LOG TRANSFORM SPECTROGRAM
    spec_images = np.clip(spec_images,np.exp(-4),np.exp(8))
    spec_images = np.log(spec_images)
    
    spec_images = np.nan_to_num(spec_images, nan=0.0)    

    ep = 1e-6
    rows = spec_images.shape[0]
    for ij in range(0, rows):
        minv = spec_images[ij].flatten().min()
        maxv = spec_images[ij].flatten().max()
        spec_images[ij] = 255 * (spec_images[ij] - minv) / (maxv - minv + ep)
        
    return spec_images

if not RUN_TRAINED_MODEL:
    spectrograms_images_c3 = egg_spec_img_normalization(spectrograms_images_c3)


Normaliztion spectrogram values to 0 ... 255


In [13]:
print('Set Up model EfficientNet')
import tensorflow as tf
import keras
from keras import layers, layers, models
from keras.applications import EfficientNetB2

def build_model_EfficientNet():
    
    with strategy.scope():
        inputs = layers.Input(shape=(128, 200, 3))
        model = EfficientNetB2(include_top=False, input_tensor=inputs)

        # Rebuild top
        x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
        #x = layers.BatchNormalization()(x)
        #top_dropout_rate = 0.2
        #x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
        outputs = layers.Dense(6, activation="softmax")(x)

        model = keras.Model(inputs, outputs, name="EfficientNet")
        optimizer = keras.optimizers.Adam(learning_rate=1e-4)
        model.compile(optimizer=optimizer,
                          loss='kullback_leibler_divergence',
                          #metrics=["accuracy"]
                          metrics=["categorical_accuracy"]
                          #metrics=[keras.metrics.KLDivergence()]
        )
    return model

if not RUN_TRAINED_MODEL:
    modelEfficientNet = build_model_EfficientNet()
    modelEfficientNet.summary()

Set Up model EfficientNet
Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 200, 3)]        0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, 128, 200, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 128, 200, 3)          7         ['rescaling_2[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 128, 200, 3)          0  

                                                                                                  
 block2a_expand_conv (Conv2  (None, 64, 100, 96)          1536      ['block1b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block2a_expand_bn (BatchNo  (None, 64, 100, 96)          384       ['block2a_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block2a_expand_activation   (None, 64, 100, 96)          0         ['block2a_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block2a_d

 block2c_expand_conv (Conv2  (None, 32, 50, 144)          3456      ['block2b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block2c_expand_bn (BatchNo  (None, 32, 50, 144)          576       ['block2c_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block2c_expand_activation   (None, 32, 50, 144)          0         ['block2c_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block2c_dwconv (DepthwiseC  (None, 32, 50, 144)          1296      ['block2c_expand_activation[0]
 onv2D)   

 D)                                                                                               
                                                                                                  
 block3b_expand_bn (BatchNo  (None, 16, 25, 288)          1152      ['block3b_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block3b_expand_activation   (None, 16, 25, 288)          0         ['block3b_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block3b_dwconv (DepthwiseC  (None, 16, 25, 288)          7200      ['block3b_expand_activation[0]
 onv2D)                                                             [0]']                         
          

 block4a_expand_conv (Conv2  (None, 16, 25, 288)          13824     ['block3c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block4a_expand_bn (BatchNo  (None, 16, 25, 288)          1152      ['block4a_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block4a_expand_activation   (None, 16, 25, 288)          0         ['block4a_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block4a_dwconv_pad (ZeroPa  (None, 17, 27, 288)          0         ['block4a_expand_activation[0]
 dding2D) 

...

 rmalization)                                                                                     
                                                                                                  
 block5c_expand_activation   (None, 8, 13, 720)           0         ['block5c_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block5c_dwconv (DepthwiseC  (None, 8, 13, 720)           18000     ['block5c_expand_activation[0]
 onv2D)                                                             [0]']                         
                                                                                                  
 block5c_bn (BatchNormaliza  (None, 8, 13, 720)           2880      ['block5c_dwconv[0][0]']      
 tion)                                                                                            
          

 block6a_expand_bn (BatchNo  (None, 8, 13, 720)           2880      ['block6a_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6a_expand_activation   (None, 8, 13, 720)           0         ['block6a_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block6a_dwconv_pad (ZeroPa  (None, 11, 17, 720)          0         ['block6a_expand_activation[0]
 dding2D)                                                           [0]']                         
                                                                                                  
 block6a_dwconv (DepthwiseC  (None, 4, 7, 720)            18000     ['block6a_dwconv_pad[0][0]']  
 onv2D)   

 rmalization)                                                                                     
                                                                                                  
 block6c_expand_activation   (None, 4, 7, 1248)           0         ['block6c_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block6c_dwconv (DepthwiseC  (None, 4, 7, 1248)           31200     ['block6c_expand_activation[0]
 onv2D)                                                             [0]']                         
                                                                                                  
 block6c_bn (BatchNormaliza  (None, 4, 7, 1248)           4992      ['block6c_dwconv[0][0]']      
 tion)                                                                                            
          

 block6e_expand_bn (BatchNo  (None, 4, 7, 1248)           4992      ['block6e_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6e_expand_activation   (None, 4, 7, 1248)           0         ['block6e_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block6e_dwconv (DepthwiseC  (None, 4, 7, 1248)           31200     ['block6e_expand_activation[0]
 onv2D)                                                             [0]']                         
                                                                                                  
 block6e_bn (BatchNormaliza  (None, 4, 7, 1248)           4992      ['block6e_dwconv[0][0]']      
 tion)    

 (Activation)                                                                                     
                                                                                                  
 block7b_dwconv (DepthwiseC  (None, 4, 7, 2112)           19008     ['block7b_expand_activation[0]
 onv2D)                                                             [0]']                         
                                                                                                  
 block7b_bn (BatchNormaliza  (None, 4, 7, 2112)           8448      ['block7b_dwconv[0][0]']      
 tion)                                                                                            
                                                                                                  
 block7b_activation (Activa  (None, 4, 7, 2112)           0         ['block7b_bn[0][0]']          
 tion)                                                                                            
          

In [14]:
print('Set Up model ResNet')
import tensorflow as tf
import keras
from keras import layers, layers, models
from keras.applications import ResNet101

def build_model_ResNet():
    
    with strategy.scope():
        inputs = layers.Input(shape=(128, 200, 3))
        model = ResNet101(include_top=False, input_tensor=inputs)

        # Rebuild top
        x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
        #x = layers.BatchNormalization()(x)
        #top_dropout_rate = 0.2
        #x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
        outputs = layers.Dense(6, activation="softmax")(x)

        model = keras.Model(inputs, outputs, name="ResNet50")
        optimizer = keras.optimizers.Adam(learning_rate=1e-4)
        model.compile(optimizer=optimizer,
                          loss='kullback_leibler_divergence',
                          #metrics=["accuracy"]
                          metrics=["categorical_accuracy"]
                          #metrics=[keras.metrics.KLDivergence()]
        )
    return model

if not RUN_TRAINED_MODEL:
    modelResNet = build_model_ResNet()
    modelResNet.summary()

Set Up model ResNet
Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128, 200, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 206, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 100, 64)          9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 100, 64)          256       ['conv1_conv[0][0]']          
 on)                                                                   

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 32, 50, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 32, 50, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 32, 50, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 16, 25, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 16, 25, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 16, 25, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 8, 13, 256)           590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 8, 13, 256)           1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 8, 13, 256)           0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 8, 13, 1024)          525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 8, 13, 1024)          0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 8, 13, 256)           262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 8, 13, 256)           1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

...

 ion)                                                                                             
                                                                                                  
 conv4_block15_1_conv (Conv  (None, 8, 13, 256)           262400    ['conv4_block14_out[0][0]']   
 2D)                                                                                              
                                                                                                  
 conv4_block15_1_bn (BatchN  (None, 8, 13, 256)           1024      ['conv4_block15_1_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block15_1_relu (Acti  (None, 8, 13, 256)           0         ['conv4_block15_1_bn[0][0]']  
 vation)                                                                                          
          

 conv4_block17_3_bn (BatchN  (None, 8, 13, 1024)          4096      ['conv4_block17_3_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block17_add (Add)     (None, 8, 13, 1024)          0         ['conv4_block16_out[0][0]',   
                                                                     'conv4_block17_3_bn[0][0]']  
                                                                                                  
 conv4_block17_out (Activat  (None, 8, 13, 1024)          0         ['conv4_block17_add[0][0]']   
 ion)                                                                                             
                                                                                                  
 conv4_block18_1_conv (Conv  (None, 8, 13, 256)           262400    ['conv4_block17_out[0][0]']   
 2D)      

                                                                                                  
 conv4_block20_2_relu (Acti  (None, 8, 13, 256)           0         ['conv4_block20_2_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv4_block20_3_conv (Conv  (None, 8, 13, 1024)          263168    ['conv4_block20_2_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block20_3_bn (BatchN  (None, 8, 13, 1024)          4096      ['conv4_block20_3_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_blo

 vation)                                                                                          
                                                                                                  
 conv4_block23_2_conv (Conv  (None, 8, 13, 256)           590080    ['conv4_block23_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block23_2_bn (BatchN  (None, 8, 13, 256)           1024      ['conv4_block23_2_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block23_2_relu (Acti  (None, 8, 13, 256)           0         ['conv4_block23_2_bn[0][0]']  
 vation)                                                                                          
          

 conv5_block2_add (Add)      (None, 4, 7, 2048)           0         ['conv5_block1_out[0][0]',    
                                                                     'conv5_block2_3_bn[0][0]']   
                                                                                                  
 conv5_block2_out (Activati  (None, 4, 7, 2048)           0         ['conv5_block2_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv5_block3_1_conv (Conv2  (None, 4, 7, 512)            1049088   ['conv5_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv5_block3_1_bn (BatchNo  (None, 4, 7, 512)            2048      ['conv5_block3_1_conv[0][0]'] 
 rmalizati

In [15]:
print('Set Up model DenseNet')
import tensorflow as tf
import keras
from keras import layers, layers, models
from keras.applications import DenseNet121

def build_model_DenseNet():
    
    with strategy.scope():
        inputs = layers.Input(shape=(128, 200, 3))
        model = DenseNet121(include_top=False, input_tensor=inputs)

        # Rebuild top
        x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
        #x = layers.BatchNormalization()(x)
        #top_dropout_rate = 0.2
        #x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
        outputs = layers.Dense(6, activation="softmax")(x)

        model = keras.Model(inputs, outputs, name="DenseNet121")
        optimizer = keras.optimizers.Adam(learning_rate=1e-4)
        model.compile(optimizer=optimizer,
                          loss='kullback_leibler_divergence',
                          #metrics=["accuracy"]
                          metrics=["categorical_accuracy"]
                          #metrics=[keras.metrics.KLDivergence()]
        )
    return model

if not RUN_TRAINED_MODEL:
    modelDenseNet = build_model_DenseNet()
    modelDenseNet.summary()

Set Up model DenseNet
Model: "DenseNet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 128, 200, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 134, 206, 3)          0         ['input_4[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 64, 100, 64)          9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 64, 100, 64)          256     

 conv2_block4_0_bn (BatchNo  (None, 32, 50, 160)          640       ['conv2_block3_concat[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block4_0_relu (Activ  (None, 32, 50, 160)          0         ['conv2_block4_0_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_block4_1_conv (Conv2  (None, 32, 50, 128)          20480     ['conv2_block4_0_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv2_block4_1_bn (BatchNo  (None, 32, 50, 128)          512       ['conv2_block4_1_conv[0][0]'] 
 rmalizati

 rmalization)                                                                                     
                                                                                                  
 conv3_block1_1_relu (Activ  (None, 16, 25, 128)          0         ['conv3_block1_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv3_block1_2_conv (Conv2  (None, 16, 25, 32)           36864     ['conv3_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block1_concat (Conca  (None, 16, 25, 160)          0         ['pool2_pool[0][0]',          
 tenate)                                                             'conv3_block1_2_conv[0][0]'] 
          

 conv3_block5_1_bn (BatchNo  (None, 16, 25, 128)          512       ['conv3_block5_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block5_1_relu (Activ  (None, 16, 25, 128)          0         ['conv3_block5_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv3_block5_2_conv (Conv2  (None, 16, 25, 32)           36864     ['conv3_block5_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block5_concat (Conca  (None, 16, 25, 288)          0         ['conv3_block4_concat[0][0]', 
 tenate)  

                                                                                                  
 conv3_block9_1_bn (BatchNo  (None, 16, 25, 128)          512       ['conv3_block9_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block9_1_relu (Activ  (None, 16, 25, 128)          0         ['conv3_block9_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv3_block9_2_conv (Conv2  (None, 16, 25, 32)           36864     ['conv3_block9_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_blo

...

 conv4_block24_1_relu (Acti  (None, 8, 12, 128)           0         ['conv4_block24_1_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv4_block24_2_conv (Conv  (None, 8, 12, 32)            36864     ['conv4_block24_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block24_concat (Conc  (None, 8, 12, 1024)          0         ['conv4_block23_concat[0][0]',
 atenate)                                                            'conv4_block24_2_conv[0][0]']
                                                                                                  
 pool4_bn (BatchNormalizati  (None, 8, 12, 1024)          4096      ['conv4_block24_concat[0][0]']
 on)      

 rmalization)                                                                                     
                                                                                                  
 conv5_block4_0_relu (Activ  (None, 4, 6, 608)            0         ['conv5_block4_0_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block4_1_conv (Conv2  (None, 4, 6, 128)            77824     ['conv5_block4_0_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block4_1_bn (BatchNo  (None, 4, 6, 128)            512       ['conv5_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
          

 conv5_block8_0_bn (BatchNo  (None, 4, 6, 736)            2944      ['conv5_block7_concat[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block8_0_relu (Activ  (None, 4, 6, 736)            0         ['conv5_block8_0_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block8_1_conv (Conv2  (None, 4, 6, 128)            94208     ['conv5_block8_0_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block8_1_bn (BatchNo  (None, 4, 6, 128)            512       ['conv5_block8_1_conv[0][0]'] 
 rmalizati

                                                                                                  
 conv5_block12_0_bn (BatchN  (None, 4, 6, 864)            3456      ['conv5_block11_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv5_block12_0_relu (Acti  (None, 4, 6, 864)            0         ['conv5_block12_0_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv5_block12_1_conv (Conv  (None, 4, 6, 128)            110592    ['conv5_block12_0_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv5_blo

 atenate)                                                            'conv5_block15_2_conv[0][0]']
                                                                                                  
 conv5_block16_0_bn (BatchN  (None, 4, 6, 992)            3968      ['conv5_block15_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv5_block16_0_relu (Acti  (None, 4, 6, 992)            0         ['conv5_block16_0_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv5_block16_1_conv (Conv  (None, 4, 6, 128)            126976    ['conv5_block16_0_relu[0][0]']
 2D)                                                                                              
          

In [16]:
%%time
from sklearn.model_selection import train_test_split, KFold
import math as ops

def train_split(spectrograms_images_c3):
    X_train, X_test, y_train, y_test = train_test_split(spectrograms_images_c3, spectrograms_labels, test_size = 0.1)

    del spectrograms_images_c3
    gc.collect()
    
    return X_train, X_test, y_train, y_test

def train_model(model, X_train, y_train):
    folds = list(KFold(n_splits=5, shuffle=True, random_state=1).split(X_train, y_train))
    
    
    def scheduler(epoch):
        return [1e-4,1e-4,1e-4,1e-5,1e-5][epoch]
            
    callback = keras.callbacks.LearningRateScheduler(scheduler)
    
    epochs = 3
    batch_size = 10
    for j, (train_index, val_index) in enumerate(folds):

        print('\nFold ',j)
        #X_train_cv = X_train[train_index]
        #y_train_cv = y_train[train_index]
        #X_valid_cv = X_train[val_index]
        #y_valid_cv = y_train[val_index]

        model.fit(
                    X_train[train_index], y_train[train_index],
                    steps_per_epoch=len(X_train[train_index])/batch_size,
                    epochs=epochs,
                    batch_size=batch_size, 
                    shuffle=True,
                    verbose=1,
                    #callbacks=[callback],
                    validation_data = (X_train[val_index], y_train[val_index])
                 )
        print(model.evaluate(X_train[val_index], y_train[val_index]))
        
    return model

    
if not RUN_TRAINED_MODEL:
    X_train, X_test, y_train, y_test = train_split(spectrograms_images_c3)
    
    modelEfficientNet = train_model(modelEfficientNet, X_train, y_train)
    if not FOR_SUBMISSION: 
        modelEfficientNet.save('modelEfficientNet.keras')
        
    modelResNet = train_model(modelResNet, X_train, y_train)    
    if not FOR_SUBMISSION: 
        modelResNet.save('modelResNet.keras')
        
    modelDenseNet = train_model(modelDenseNet, X_train, y_train)    
    if not FOR_SUBMISSION: 
        modelDenseNet.save('modelDenseNet.keras')



Fold  0
Epoch 1/3


332/332 [==============================] - 227s 597ms/step - loss: 0.6964 - categorical_accuracy: 0.5679 - val_loss: 0.5150 - val_categorical_accuracy: 0.6619
Epoch 2/3
332/332 [==============================] - 197s 592ms/step - loss: 0.4563 - categorical_accuracy: 0.6894 - val_loss: 0.4769 - val_categorical_accuracy: 0.6775
Epoch 3/3
26/26 [==============================] - 8s 316ms/step - loss: 0.4749 - categorical_accuracy: 0.6847
[0.4749491810798645, 0.6847172379493713]

Fold  1
Epoch 1/3
332/332 [==============================] - 196s 590ms/step - loss: 0.3664 - categorical_accuracy: 0.7367 - val_loss: 0.2332 - val_categorical_accuracy: 0.8375
Epoch 2/3
332/332 [==============================] - 191s 574ms/step - loss: 0.3081 - categorical_accuracy: 0.7728 - val_loss: 0.2367 - val_categorical_accuracy: 0.8171
Epoch 3/3
26/26 [==============================] - 8s 305ms/step - loss: 0.2490 - categorical_accuracy: 0.8123
[0.24902881681919098, 0.8122743964195251]

KeyboardInterrupt: 

In [66]:
if not RUN_TRAINED_MODEL:
    pred_test1 = modelResNet.predict(X_test)
    kl = tf.keras.losses.KLDivergence()
    kl(y_test, pred_test1).numpy()
    print(kl(y_test, pred_test1).numpy())

15/15 [==============================] - 18s 1s/step
0.40891686


In [64]:
if not RUN_TRAINED_MODEL:
    pred_test2 = modelEfficientNet.predict(X_test)
    kl = tf.keras.losses.KLDivergence()
    kl(y_test, pred_test2).numpy()
    print(kl(y_test, pred_test2).numpy())

15/15 [==============================] - 8s 333ms/step
0.37157166


In [ ]:
if not RUN_TRAINED_MODEL:
    pred_test3 = modelDenseNet.predict(X_test)
    kl = tf.keras.losses.KLDivergence()
    kl(y_test, pred_test3).numpy()
    print(kl(y_test, pred_test3).numpy())

In [67]:
if not RUN_TRAINED_MODEL:
    pred_test = (pred_test1 + pred_test2 + pred_test3)/3.0
    kl = tf.keras.losses.KLDivergence()
    print(kl(y_test, pred_test).numpy())

0.3346764


In [15]:
%%time
%%time
if RUN_ON_KAGGLE:
    TEST_SPEC_PATH = FILE_BASE_PATH + "test_spectrograms/"
else:
    TEST_SPEC_PATH = FILE_BASE_PATH + "test_spectrograms\\"
files = os.listdir(TEST_SPEC_PATH)
print(f'There are {len(files)} spectrogram parquets')

test_spectrograms_ids = np.empty((0), dtype = np.int64)
test_spectrograms_images = np.empty((0, 256, 400))
test_spectrograms_labels = np.empty((0,6))
test_spectrograms_experts = np.empty((0), dtype = np.string_)
for i,f in enumerate(files):
    if i%100==0:print(i,', ',end='')
    name = int(f.split('.')[0])
    tmp = pd.read_parquet(f'{TEST_SPEC_PATH}{f}')
    test_spectrograms_images = np.insert(test_spectrograms_images, test_spectrograms_images.shape[0], 
                                                tmp.iloc[0:256,1:].values, axis=0)
    test_spectrograms_ids = np.insert(test_spectrograms_ids, test_spectrograms_ids.shape[0], [name], axis=0)
    

There are 1 spectrogram parquets
0 , CPU times: user 115 ms, sys: 16.1 ms, total: 131 ms
Wall time: 337 ms
CPU times: user 120 ms, sys: 16.9 ms, total: 136 ms
Wall time: 342 ms


In [16]:
test_spectrograms_images_c3 = egg_spec_img_resize(test_spectrograms_images)
test_spectrograms_images_c3 = egg_spec_img_normalization(test_spectrograms_images_c3)

In [17]:
if RUN_TRAINED_MODEL:
    if RUN_ON_KAGGLE:
        MODEL_PATH = MY_FILE_BASE_PATH
    else:
        MODEL_PATH = MY_FILE_BASE_PATH
    print('Load model modelResNet')
    modelResNet = tf.keras.models.load_model(MODEL_PATH + 'modelResNet.keras')
    pred_test_spectrograms_images_c3_1 = modelResNet.predict(test_spectrograms_images_c3)
    
    print('Load model modelEfficientNet')
    modelEfficientNet = tf.keras.models.load_model(MODEL_PATH + 'modelEfficientNet.keras')
    pred_test_spectrograms_images_c3_2 = modelEfficientNet.predict(test_spectrograms_images_c3)
    
    pred_test_spectrograms_images_c3 = (pred_test_spectrograms_images_c3_1 + pred_test_spectrograms_images_c3_2)/2.0
    
import random

test = pd.read_csv(FILE_BASE_PATH + 'test.csv')

eeg_ids = np.empty((0), dtype = np.int64)
for id in test_spectrograms_ids:
    eeg_id = test[test['spectrogram_id'] == id]['eeg_id']
    if eeg_id.shape[0] > 0:
        eeg_ids = np.insert(eeg_ids, eeg_ids.shape[0], [eeg_id.values[0]], axis = 0)
    
print('eeg_ids.shape=', eeg_ids.shape)

submission = pd.DataFrame({'eeg_id':eeg_ids})

pred_test_spectrograms_images_c3 = np.nan_to_num(pred_test_spectrograms_images_c3, nan=0.0)

submission[TARGETS] = pred_test_spectrograms_images_c3
submission.to_csv('submission.csv',index=False)
print('submissionn',submission.shape)
print(submission.head())

1/1 [==============================] - 1s 549ms/step
pred_test_spectrograms_images_c3 shape (1, 6)
test_spectrograms_images_c3 shape (1, 128, 200, 3)
eeg_ids.shape= (1,)
submissionn (1, 7)
       eeg_id  seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  \
0  3911565283      0.001065    0.0741  0.000001    0.32504   0.039786   

   other_vote  
0    0.560007  
